In [1]:
pip install lxml pyproj numpy

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 5.8 MB/s eta 0:00:01
   ----------- ---------------------------- 1.8/6.3 MB 4.7 MB/s eta 0:00:01
   ----------------------- ---------------- 3.7/6.3 MB 6.3 MB/s eta 0:00:01
   ---------------------------------- ----- 5.5/6.3 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xml.etree.ElementTree as ET
from lxml import etree
import csv
import numpy as np
from pyproj import Transformer
import os

class CityGMLParser:
    def __init__(self, input_file, output_csv):
        self.input_file = input_file
        self.output_csv = output_csv
        
        # Common CityGML namespaces
        self.namespaces = {
            'core': 'http://www.opengis.net/citygml/2.0',
            'bldg': 'http://www.opengis.net/citygml/building/2.0',
            'gml': 'http://www.opengis.net/gml',
            'gen': 'http://www.opengis.net/citygml/generics/2.0'
        }
        
        # Transformer from NAD83 (EPSG:2950 for Montreal) to WGS84 (lat/lon)
        # Adjust EPSG code based on your data's coordinate system
        self.transformer = Transformer.from_crs("EPSG:2950", "EPSG:4326", always_xy=True)
        
    def parse_pos_list(self, pos_list_text):
        """Parse a gml:posList into coordinate tuples"""
        coords = list(map(float, pos_list_text.split()))
        # Group into (x, y, z) tuples
        points = [(coords[i], coords[i+1], coords[i+2] if i+2 < len(coords) else 0) 
                  for i in range(0, len(coords), 3)]
        return points
    
    def calculate_centroid(self, points):
        """Calculate the centroid of a set of points"""
        if not points:
            return None
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        z_coords = [p[2] for p in points]
        return (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    
    def calculate_convex_hull_2d(self, points):
        """Calculate 2D convex hull (simplified for building footprint)"""
        # Use Graham scan algorithm for 2D points
        points_2d = [(p[0], p[1]) for p in points]
        points_2d = list(set(points_2d))  # Remove duplicates
        
        if len(points_2d) < 3:
            return points_2d
        
        # Sort points
        points_2d = sorted(points_2d, key=lambda p: (p[0], p[1]))
        
        # Simple convex hull (for small datasets)
        # For production, use scipy.spatial.ConvexHull
        return points_2d[:min(8, len(points_2d))]  # Simplified - return first 8 points
    
    def calculate_dimensions(self, points):
        """Calculate latitudinal and longitudinal length"""
        if not points:
            return 0, 0
        
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        
        lat_length = max(x_coords) - min(x_coords)
        lon_length = max(y_coords) - min(y_coords)
        
        return lat_length, lon_length
    
    def extract_building_data(self):
        """Extract building data from CityGML file"""
        buildings = []
        
        try:
            # Parse the XML file
            tree = etree.parse(self.input_file)
            root = tree.getroot()
            
            # Find all building elements
            building_elements = root.xpath('//bldg:Building', namespaces=self.namespaces)
            
            if not building_elements:
                # Try alternative namespace approach
                building_elements = root.findall('.//{http://www.opengis.net/citygml/building/2.0}Building')
            
            print(f"Found {len(building_elements)} buildings")
            
            for idx, building in enumerate(building_elements):
                try:
                    # Extract building ID
                    building_id = building.get('{http://www.opengis.net/gml}id', f'Building_{idx}')
                    
                    # Extract all coordinate points from the building
                    all_points = []
                    ground_points = []
                    
                    # Find all posList elements
                    pos_lists = building.xpath('.//gml:posList', namespaces=self.namespaces)
                    
                    if not pos_lists:
                        pos_lists = building.findall('.//{http://www.opengis.net/gml}posList')
                    
                    for pos_list in pos_lists:
                        if pos_list.text:
                            points = self.parse_pos_list(pos_list.text)
                            all_points.extend(points)
                            
                            # Check if this is ground surface
                            parent = pos_list.getparent()
                            if parent is not None:
                                parent_text = etree.tostring(parent, encoding='unicode')
                                if 'GroundSurface' in parent_text or 'ground' in parent_text.lower():
                                    ground_points.extend(points)
                    
                    if not all_points:
                        print(f"Warning: No coordinates found for {building_id}")
                        continue
                    
                    # Use ground points if available, otherwise use all points
                    footprint_points = ground_points if ground_points else all_points
                    
                    # Calculate building properties
                    centroid = self.calculate_centroid(footprint_points)
                    convex_hull = self.calculate_convex_hull_2d(footprint_points)
                    lat_length, lon_length = self.calculate_dimensions(footprint_points)
                    
                    # Get height (z-coordinate range)
                    z_coords = [p[2] for p in all_points]
                    height = max(z_coords) - min(z_coords) if z_coords else 0
                    
                    # Transform coordinates to lat/lon
                    centroid_latlon = self.transformer.transform(centroid[0], centroid[1])
                    
                    # Format ground surface points
                    ground_surface_str = ';'.join([f"{p[0]},{p[1]},{p[2]}" for p in footprint_points[:20]])  # Limit to 20 points
                    convex_hull_str = ';'.join([f"{p[0]},{p[1]}" for p in convex_hull])
                    
                    building_data = {
                        'BuildingID': building_id,
                        'Centroid_X': centroid[0],
                        'Centroid_Y': centroid[1],
                        'Centroid_Z': centroid[2],
                        'Centroid_Lat': centroid_latlon[1],
                        'Centroid_Lon': centroid_latlon[0],
                        'Height': height,
                        'LatitudinalLength': lat_length,
                        'LongitudinalLength': lon_length,
                        'GroundSurfacePoints': ground_surface_str,
                        'ConvexHullPoints': convex_hull_str,
                        'NumVertices': len(all_points)
                    }
                    
                    buildings.append(building_data)
                    print(f"Processed: {building_id}")
                    
                except Exception as e:
                    print(f"Error processing building {idx}: {str(e)}")
                    continue
            
        except Exception as e:
            print(f"Error parsing CityGML file: {str(e)}")
            raise
        
        return buildings
    
    def write_csv(self, buildings):
        """Write building data to CSV file"""
        if not buildings:
            print("No buildings to write!")
            return
        
        fieldnames = [
            'BuildingID', 'Centroid_X', 'Centroid_Y', 'Centroid_Z',
            'Centroid_Lat', 'Centroid_Lon', 'Height',
            'LatitudinalLength', 'LongitudinalLength',
            'GroundSurfacePoints', 'ConvexHullPoints', 'NumVertices'
        ]
        
        with open(self.output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(buildings)
        
        print(f"Successfully wrote {len(buildings)} buildings to {self.output_csv}")
    
    def process(self):
        """Main processing function"""
        print(f"Processing CityGML file: {self.input_file}")
        buildings = self.extract_building_data()
        self.write_csv(buildings)
        return buildings




In [5]:
import xml.etree.ElementTree as ET
from lxml import etree
import csv
import numpy as np
from pyproj import Transformer
import os

class CityGMLParser:
    def __init__(self, input_file, output_csv):
        self.input_file = input_file
        self.output_csv = output_csv
        
        # CityGML 1.0 namespaces (matching your file format)
        self.namespaces = {
            'citygml': 'http://www.opengis.net/citygml/1.0',
            'bldg': 'http://www.opengis.net/citygml/building/1.0',
            'gml': 'http://www.opengis.net/gml',
            'gen': 'http://www.opengis.net/citygml/generics/1.0',
            'app': 'http://www.opengis.net/citygml/appearance/1.0'
        }
        
        # Transformer from NAD83 (EPSG:2950 for Montreal) to WGS84 (lat/lon)
        # Adjust EPSG code based on your data's coordinate system
        self.transformer = Transformer.from_crs("EPSG:2950", "EPSG:4326", always_xy=True)
        
    def parse_pos_list(self, pos_list_text):
        """Parse a gml:posList into coordinate tuples"""
        coords = list(map(float, pos_list_text.split()))
        # Group into (x, y, z) tuples
        points = [(coords[i], coords[i+1], coords[i+2] if i+2 < len(coords) else 0) 
                  for i in range(0, len(coords), 3)]
        return points
    
    def calculate_centroid(self, points):
        """Calculate the centroid of a set of points"""
        if not points:
            return None
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        z_coords = [p[2] for p in points]
        return (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    
    def calculate_convex_hull_2d(self, points):
        """Calculate 2D convex hull (simplified for building footprint)"""
        # Use Graham scan algorithm for 2D points
        points_2d = [(p[0], p[1]) for p in points]
        points_2d = list(set(points_2d))  # Remove duplicates
        
        if len(points_2d) < 3:
            return points_2d
        
        # Sort points
        points_2d = sorted(points_2d, key=lambda p: (p[0], p[1]))
        
        # Simple convex hull (for small datasets)
        # For production, use scipy.spatial.ConvexHull
        return points_2d[:min(8, len(points_2d))]  # Simplified - return first 8 points
    
    def calculate_dimensions(self, points):
        """Calculate latitudinal and longitudinal length"""
        if not points:
            return 0, 0
        
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        
        lat_length = max(x_coords) - min(x_coords)
        lon_length = max(y_coords) - min(y_coords)
        
        return lat_length, lon_length
    
    def extract_building_data(self):
        """Extract building data from CityGML file"""
        buildings = []
        
        try:
            # Parse the XML file
            tree = etree.parse(self.input_file)
            root = tree.getroot()
            
            # Find all building elements (CityGML 1.0 format)
            building_elements = root.xpath('//bldg:Building', namespaces=self.namespaces)
            
            if not building_elements:
                # Try direct namespace approach
                building_elements = root.findall('.//{http://www.opengis.net/citygml/building/1.0}Building')
            
            print(f"Found {len(building_elements)} buildings")
            
            for idx, building in enumerate(building_elements):
                try:
                    # Extract building ID (from gml:id attribute)
                    building_id = building.get('{http://www.opengis.net/gml}id', f'Building_{idx}')
                    
                    # Extract volume attribute if available
                    volume = 0
                    volume_elem = building.find('.//gen:doubleAttribute[@name="Volume"]/gen:value', namespaces=self.namespaces)
                    if volume_elem is not None and volume_elem.text:
                        try:
                            volume = float(volume_elem.text)
                        except:
                            pass
                    
                    # Extract all coordinate points from the building
                    all_points = []
                    ground_points = []
                    roof_points = []
                    
                    # Find all posList elements
                    pos_lists = building.xpath('.//gml:posList', namespaces=self.namespaces)
                    
                    if not pos_lists:
                        pos_lists = building.findall('.//{http://www.opengis.net/gml}posList')
                    
                    for pos_list in pos_lists:
                        if pos_list.text:
                            points = self.parse_pos_list(pos_list.text)
                            all_points.extend(points)
                            
                            # Check if this is ground surface or roof surface
                            parent = pos_list.getparent()
                            ancestors = []
                            current = parent
                            # Get all ancestors up to 5 levels
                            for _ in range(5):
                                if current is not None:
                                    ancestors.append(current)
                                    current = current.getparent()
                                else:
                                    break
                            
                            # Check ancestor elements for surface type
                            is_ground = False
                            is_roof = False
                            for ancestor in ancestors:
                                tag = ancestor.tag
                                if 'GroundSurface' in tag:
                                    is_ground = True
                                    break
                                elif 'RoofSurface' in tag:
                                    is_roof = True
                                    break
                            
                            if is_ground:
                                ground_points.extend(points)
                            elif is_roof:
                                roof_points.extend(points)
                    
                    if not all_points:
                        print(f"Warning: No coordinates found for {building_id}")
                        continue
                    
                    # Use ground points if available, otherwise use all points
                    footprint_points = ground_points if ground_points else all_points
                    
                    # Calculate building properties
                    centroid = self.calculate_centroid(footprint_points)
                    convex_hull = self.calculate_convex_hull_2d(footprint_points)
                    lat_length, lon_length = self.calculate_dimensions(footprint_points)
                    
                    # Get height (z-coordinate range)
                    z_coords = [p[2] for p in all_points]
                    height = max(z_coords) - min(z_coords) if z_coords else 0
                    
                    # Transform coordinates to lat/lon
                    centroid_latlon = self.transformer.transform(centroid[0], centroid[1])
                    
                    # Format ground surface points
                    ground_surface_str = ';'.join([f"{p[0]},{p[1]},{p[2]}" for p in footprint_points[:20]])  # Limit to 20 points
                    convex_hull_str = ';'.join([f"{p[0]},{p[1]}" for p in convex_hull])
                    
                    building_data = {
                        'BuildingID': building_id,
                        'Centroid_X': centroid[0],
                        'Centroid_Y': centroid[1],
                        'Centroid_Z': centroid[2],
                        'Centroid_Lat': centroid_latlon[1],
                        'Centroid_Lon': centroid_latlon[0],
                        'Height': height,
                        'Volume': volume,
                        'LatitudinalLength': lat_length,
                        'LongitudinalLength': lon_length,
                        'GroundSurfacePoints': ground_surface_str,
                        'ConvexHullPoints': convex_hull_str,
                        'NumVertices': len(all_points),
                        'NumRoofPoints': len(roof_points),
                        'NumGroundPoints': len(ground_points)
                    }
                    
                    buildings.append(building_data)
                    print(f"Processed: {building_id}")
                    
                except Exception as e:
                    print(f"Error processing building {idx}: {str(e)}")
                    continue
            
        except Exception as e:
            print(f"Error parsing CityGML file: {str(e)}")
            raise
        
        return buildings
    
    def write_csv(self, buildings):
        """Write building data to CSV file"""
        if not buildings:
            print("No buildings to write!")
            return
        
        fieldnames = [
            'BuildingID', 'Centroid_X', 'Centroid_Y', 'Centroid_Z',
            'Centroid_Lat', 'Centroid_Lon', 'Height', 'Volume',
            'LatitudinalLength', 'LongitudinalLength',
            'GroundSurfacePoints', 'ConvexHullPoints', 'NumVertices',
            'NumRoofPoints', 'NumGroundPoints'
        ]
        
        with open(self.output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(buildings)
        
        print(f"Successfully wrote {len(buildings)} buildings to {self.output_csv}")
    
    def process(self):
        """Main processing function"""
        print(f"Processing CityGML file: {self.input_file}")
        buildings = self.extract_building_data()
        self.write_csv(buildings)
        return buildings




In [6]:
# Usage example
if __name__ == "__main__":
    # Configure your file paths
    input_gml = "C:/Users/Isaac IK/Downloads/vm182013/VM18_2013.gml"  # Change this to your GML file path
    output_csv = "buildings_for_anylogic.csv"     # Output CSV file
    
    # Create parser and process
    parser = CityGMLParser(input_gml, output_csv)
    
    try:
        buildings = parser.process()
        print(f"\nSummary:")
        print(f"Total buildings processed: {len(buildings)}")
        print(f"Output file: {output_csv}")
        
        # Print sample of first building
        if buildings:
            print("\nSample building data:")
            for key, value in list(buildings[0].items())[:6]:
                print(f"  {key}: {value}")
                
    except Exception as e:
        print(f"Failed to process CityGML file: {str(e)}")
        print("\nTroubleshooting tips:")
        print("1. Check that the input file path is correct")
        print("2. Verify the CityGML file is valid XML")
        print("3. Adjust the EPSG code in __init__ if your data uses different coordinates")
        print("4. Install required packages: pip install lxml pyproj numpy")

Processing CityGML file: C:/Users/Isaac IK/Downloads/vm182013/VM18_2013.gml
Found 311 buildings
Processed: Groupe9365646
Processed: Groupe9365641
Processed: Groupe9365640
Processed: Groupe9365644
Processed: Groupe9365639
Processed: Groupe9365634
Processed: Groupe9365645
Processed: Groupe9365642
Processed: Groupe9365637
Processed: Groupe9365638
Processed: Groupe9365633
Processed: Groupe9365632
Processed: Groupe9365630
Processed: Groupe9365629
Processed: Groupe9365627
Processed: Groupe9365626
Processed: Groupe9365628
Processed: Groupe9365625
Processed: Groupe9365621
Processed: Groupe9365622
Processed: Groupe9365616
Processed: Groupe9365615
Processed: Groupe9365617
Processed: Groupe9365611
Processed: Groupe9365613
Processed: Groupe9365612
Processed: Groupe9365614
Processed: Groupe9365607
Processed: Groupe9365609
Processed: Groupe9365608
Processed: Groupe9365610
Processed: Groupe9365606
Processed: Groupe9365503
Processed: Groupe9365603
Processed: Groupe9365605
Processed: Groupe9365604
Proc

In [6]:
# Usage example
if __name__ == "__main__":
    # Configure your file paths
    input_gml = "C:/Users/Isaac IK/Videos/VM14_2016.gml"  # Change this to your GML file path
    output_csv = "C:/Users/Isaac IK/Videos/exported_buildings_dae/buildings_for_anylogic.csv"     # Output CSV file

    # Create parser and process
    parser = CityGMLParser(input_gml, output_csv)
    
    try:
        buildings = parser.process()
        print(f"\nSummary:")
        print(f"Total buildings processed: {len(buildings)}")
        print(f"Output file: {output_csv}")
        
        # Print sample of first building
        if buildings:
            print("\nSample building data:")
            for key, value in list(buildings[0].items())[:6]:
                print(f"  {key}: {value}")
                
    except Exception as e:
        print(f"Failed to process CityGML file: {str(e)}")
        print("\nTroubleshooting tips:")
        print("1. Check that the input file path is correct")
        print("2. Verify the CityGML file is valid XML")
        print("3. Adjust the EPSG code in __init__ if your data uses different coordinates")
        print("4. Install required packages: pip install lxml pyproj numpy")

Processing CityGML file: C:/Users/Isaac IK/Videos/VM14_2016.gml
Found 0 buildings
No buildings to write!

Summary:
Total buildings processed: 0
Output file: C:/Users/Isaac IK/Videos/exported_buildings_dae/buildings_for_anylogic.csv


In [3]:
from lxml import etree
import os
from datetime import datetime
import numpy as np

class CityGMLToDAEExporter:
    def __init__(self, input_gml, output_dir):
        self.input_gml = input_gml
        self.output_dir = output_dir
        
        # CityGML 1.0 namespaces
        self.namespaces = {
            'citygml': 'http://www.opengis.net/citygml/1.0',
            'bldg': 'http://www.opengis.net/citygml/building/1.0',
            'gml': 'http://www.opengis.net/gml',
            'gen': 'http://www.opengis.net/citygml/generics/1.0',
            'app': 'http://www.opengis.net/citygml/appearance/1.0'
        }
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
    def parse_pos_list(self, pos_list_text):
        """Parse a gml:posList into coordinate tuples"""
        coords = list(map(float, pos_list_text.split()))
        points = [(coords[i], coords[i+1], coords[i+2] if i+2 < len(coords) else 0) 
                  for i in range(0, len(coords), 3)]
        return points
    
    def calculate_normal(self, v0, v1, v2):
        """Calculate face normal from three vertices"""
        edge1 = np.array([v1[0] - v0[0], v1[1] - v0[1], v1[2] - v0[2]])
        edge2 = np.array([v2[0] - v0[0], v2[1] - v0[1], v2[2] - v0[2]])
        
        normal = np.cross(edge1, edge2)
        length = np.linalg.norm(normal)
        
        if length > 0:
            normal = normal / length
        else:
            normal = np.array([0, 0, 1])
        
        return tuple(normal)
    
    def triangulate_polygon(self, points):
        """Convert a polygon into triangles using ear clipping method"""
        if len(points) < 3:
            return []
        
        if len(points) == 3:
            return [points]
        
        # Simple fan triangulation from first vertex
        triangles = []
        for i in range(1, len(points) - 1):
            triangles.append([points[0], points[i], points[i + 1]])
        
        return triangles
    
    def extract_building_geometry(self, building_elem):
        """Extract all geometry from a building element"""
        all_triangles = []
        
        # Find all polygons in the building
        polygons = building_elem.xpath('.//gml:Polygon', namespaces=self.namespaces)
        
        for polygon in polygons:
            # Get exterior ring
            pos_lists = polygon.xpath('.//gml:posList', namespaces=self.namespaces)
            
            for pos_list in pos_lists:
                if pos_list.text:
                    points = self.parse_pos_list(pos_list.text)
                    
                    # Remove duplicate consecutive points
                    cleaned_points = [points[0]]
                    for p in points[1:]:
                        if p != cleaned_points[-1]:
                            cleaned_points.append(p)
                    
                    # Remove last point if it's same as first (closed polygon)
                    if len(cleaned_points) > 1 and cleaned_points[0] == cleaned_points[-1]:
                        cleaned_points = cleaned_points[:-1]
                    
                    # Triangulate the polygon
                    if len(cleaned_points) >= 3:
                        triangles = self.triangulate_polygon(cleaned_points)
                        all_triangles.extend(triangles)
        
        return all_triangles
    
    def create_dae_file(self, building_elem, building_id):
        """Create a complete DAE file for a single building"""
        
        # Extract geometry as triangles
        triangles = self.extract_building_geometry(building_elem)
        
        if not triangles:
            print(f"Warning: No geometry found for building {building_id}")
            return None
        
        # Flatten all vertices and calculate normals
        vertices = []
        normals = []
        indices = []
        
        current_index = 0
        for tri in triangles:
            if len(tri) == 3:
                # Add vertices
                vertices.extend(tri)
                
                # Calculate normal for this triangle
                normal = self.calculate_normal(tri[0], tri[1], tri[2])
                # Each vertex gets the same normal (flat shading)
                normals.extend([normal, normal, normal])
                
                # Add indices
                indices.extend([current_index, current_index + 1, current_index + 2])
                current_index += 3
        
        # Create DAE XML structure
        dae_content = self.build_dae_xml(building_id, vertices, normals, indices)
        
        return dae_content
    
    def build_dae_xml(self, building_id, vertices, normals, indices):
        """Build complete DAE XML content"""
        
        # Prepare vertex positions
        positions = []
        for v in vertices:
            positions.extend([v[0], v[1], v[2]])
        
        # Prepare normals
        normal_values = []
        for n in normals:
            normal_values.extend([n[0], n[1], n[2]])
        
        vertex_count = len(vertices)
        triangle_count = len(indices) // 3
        
        # Format arrays
        position_str = ' '.join(f'{x:.6f}' for x in positions)
        normal_str = ' '.join(f'{x:.6f}' for x in normal_values)
        indices_str = ' '.join(map(str, indices))
        
        # Build DAE XML
        dae = f'''<?xml version="1.0" encoding="utf-8"?>
<COLLADA xmlns="http://www.collada.org/2005/11/COLLADASchema" version="1.4.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <asset>
    <contributor>
      <author>CityGML Converter</author>
      <authoring_tool>Python</authoring_tool>
    </contributor>
    <created>{datetime.now().isoformat()}</created>
    <modified>{datetime.now().isoformat()}</modified>
    <unit name="meter" meter="1"/>
    <up_axis>Z_UP</up_axis>
  </asset>
  
  <library_geometries>
    <geometry id="{building_id}-mesh" name="{building_id}">
      <mesh>
        <source id="{building_id}-positions">
          <float_array id="{building_id}-positions-array" count="{len(positions)}">{position_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-positions-array" count="{vertex_count}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
        
        <source id="{building_id}-normals">
          <float_array id="{building_id}-normals-array" count="{len(normal_values)}">{normal_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-normals-array" count="{len(normals)}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
        
        <vertices id="{building_id}-vertices">
          <input semantic="POSITION" source="#{building_id}-positions"/>
        </vertices>
        
        <triangles material="material0" count="{triangle_count}">
          <input semantic="VERTEX" source="#{building_id}-vertices" offset="0"/>
          <input semantic="NORMAL" source="#{building_id}-normals" offset="0"/>
          <p>{indices_str}</p>
        </triangles>
      </mesh>
    </geometry>
  </library_geometries>
  
  <library_materials>
    <material id="material0" name="material0">
      <instance_effect url="#effect0"/>
    </material>
  </library_materials>
  
  <library_effects>
    <effect id="effect0">
      <profile_COMMON>
        <technique sid="common">
          <phong>
            <diffuse>
              <color>0.8 0.8 0.8 1.0</color>
            </diffuse>
            <specular>
              <color>0.2 0.2 0.2 1.0</color>
            </specular>
            <shininess>
              <float>20.0</float>
            </shininess>
          </phong>
        </technique>
      </profile_COMMON>
    </effect>
  </library_effects>
  
  <library_visual_scenes>
    <visual_scene id="Scene" name="Scene">
      <node id="{building_id}" name="{building_id}" type="NODE">
        <matrix sid="transform">1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1</matrix>
        <instance_geometry url="#{building_id}-mesh" name="{building_id}">
          <bind_material>
            <technique_common>
              <instance_material symbol="material0" target="#material0"/>
            </technique_common>
          </bind_material>
        </instance_geometry>
      </node>
    </visual_scene>
  </library_visual_scenes>
  
  <scene>
    <instance_visual_scene url="#Scene"/>
  </scene>
</COLLADA>'''
        
        return dae
    
    def export_all_buildings(self):
        """Main function to export all buildings as individual DAE files"""
        try:
            # Parse the CityGML file
            tree = etree.parse(self.input_gml)
            root = tree.getroot()
            
            # Find all building elements
            buildings = root.xpath('//bldg:Building', namespaces=self.namespaces)
            
            if not buildings:
                buildings = root.findall('.//{http://www.opengis.net/citygml/building/1.0}Building')
            
            print(f"Found {len(buildings)} buildings in the CityGML file")
            
            exported_count = 0
            failed_count = 0
            
            for idx, building in enumerate(buildings):
                # Get building ID
                building_id = building.get('{http://www.opengis.net/gml}id', f'Building_{idx}')
                
                # Sanitize filename
                safe_filename = "".join(c for c in building_id if c.isalnum() or c in ('-', '_'))
                output_file = os.path.join(self.output_dir, f"{safe_filename}.dae")
                
                print(f"Processing {idx + 1}/{len(buildings)}: {building_id}...")
                
                try:
                    # Create DAE content
                    dae_content = self.create_dae_file(building, building_id)
                    
                    if dae_content:
                        # Write to file
                        with open(output_file, 'w', encoding='utf-8') as f:
                            f.write(dae_content)
                        
                        exported_count += 1
                        print(f"  ✓ Exported to: {output_file}")
                    else:
                        failed_count += 1
                        print(f"  ✗ Failed: No geometry found")
                
                except Exception as e:
                    failed_count += 1
                    print(f"  ✗ Error: {str(e)}")
            
            # Summary
            print(f"\n{'='*60}")
            print(f"Export Summary:")
            print(f"  Total buildings: {len(buildings)}")
            print(f"  Successfully exported: {exported_count}")
            print(f"  Failed: {failed_count}")
            print(f"  Output directory: {self.output_dir}")
            print(f"{'='*60}")
            
            return exported_count, failed_count
        
        except Exception as e:
            print(f"Error processing CityGML file: {str(e)}")
            raise




In [4]:

# Usage example
if __name__ == "__main__":
    # Configure paths
    input_gml_file = "C:/Users/Isaac IK/Videos/VM14_2016.gml"  # Your CityGML file
    output_directory = "C:/Users/Isaac IK/Videos/exported_buildings_dae"  # Output folder for DAE files
    
    # Create exporter
    exporter = CityGMLToDAEExporter(input_gml_file, output_directory)
    
    # Export all buildings
    try:
        exported, failed = exporter.export_all_buildings()
        print(f"\nDone! Check the '{output_directory}' folder for your DAE files.")
        print("\nTo import into Blender:")
        print("  File → Import → Collada (.dae)")
        
    except Exception as e:
        print(f"Failed to export buildings: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Check that the input GML file path is correct")
        print("2. Ensure you have write permissions for the output directory")
        print("3. Install required packages: pip install lxml numpy")

Found 0 buildings in the CityGML file

Export Summary:
  Total buildings: 0
  Successfully exported: 0
  Failed: 0
  Output directory: C:/Users/Isaac IK/Videos/exported_buildings_dae

Done! Check the 'C:/Users/Isaac IK/Videos/exported_buildings_dae' folder for your DAE files.

To import into Blender:
  File → Import → Collada (.dae)


In [9]:
pip install osgeo

  Using cached osgeo-0.0.1.tar.gz (1.2 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for osgeo
Failed to build osgeo
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'osgeo' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'osgeo'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [73 lines of output]
  running bdist_wheel
  running build
  C:\Users\Isaac IK\anaconda3\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
  !!
  
          ********************************************************************************
          Please avoid running ``setup.py`` directly.
          Instead, use pypa/build, pypa/installer or other
          stand

In [10]:
pip install GDAL

     ---------------------------------------- 0.0/902.4 kB ? eta -:--:--
     ---------------------- --------------- 524.3/902.4 kB 3.1 MB/s eta 0:00:01
     -------------------------------------- 902.4/902.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build GDAL
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [125 lines of output]
  C:\Users\Isaac IK\AppData\Local\Temp\pip-build-env-e63wa5vo\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsWarning: `license` overwritten by `pyproject.toml`
    corresp(dist, value, root_dir)
  C:\Users\Isaac IK\AppData\Local\Temp\pip-build-env-e63wa5vo\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsWarning: `extras_require` overwritten in `pyproject.toml` (optional-dependencies)
    corresp(dist, value, root_dir)
  Using numpy 2.3.4
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-313\osgeo
  copying osgeo\gdal.py -> build\lib.win-amd64-cpython-313\osgeo
  copying osgeo\gdalconst.py -> build\lib.win-amd64-cpython-313\osgeo
  copying osgeo\gdalnumeric.py -> build\lib.win-amd64-cpython-313\osgeo
  copying osgeo\gdal_array.py -> build\lib.win-amd64-cpython-313\osgeo
  copying osgeo\gdal_f

In [1]:
from osgeo import ogr
import xml.etree.ElementTree as ET

def gml_to_dae(input_gml_path, output_dae_path):
    # Open the input GML file
    driver = ogr.GetDriverByName("GML")
    data_source = driver.Open(input_gml_path, 0)
    if data_source is None:
        print(f"Could not open {input_gml_path}")
        return

    # Create the root element for COLLADA
    root = ET.Element("COLLADA", version="1.4.1")
    asset = ET.SubElement(root, "asset")
    contributor = ET.SubElement(asset, "contributor")
    authoring_tool = ET.SubElement(contributor, "authoring_tool")
    authoring_tool.text = "Python GML to DAE Converter"

    # Iterate through layers and features
    for i in range(data_source.GetLayerCount()):
        layer = data_source.GetLayerByIndex(i)
        for feature in layer:
            geometry = feature.GetGeometryRef()
            if geometry is not None:
                geom_name = geometry.GetGeometryName()

                # Example: Handling Point geometry (very basic)
                if geom_name == "POINT":
                    x = geometry.GetX()
                    y = geometry.GetY()
                    z = geometry.GetZ() if geometry.GetZ() else 0 # Handle 2D points

                    # Create a node for the point in DAE
                    library_geometries = ET.SubElement(root, "library_geometries")
                    geometry_elem = ET.SubElement(library_geometries, "geometry", id=f"point-{feature.GetFID()}")
                    mesh = ET.SubElement(geometry_elem, "mesh")
                    source = ET.SubElement(mesh, "source", id=f"point-{feature.GetFID()}-positions")
                    float_array = ET.SubElement(source, "float_array", id=f"point-{feature.GetFID()}-positions-array", count="3")
                    float_array.text = f"{x} {y} {z}"
                    technique_common = ET.SubElement(source, "technique_common")
                    accessor = ET.SubElement(technique_common, "accessor", count="1", source=f"#point-{feature.GetFID()}-positions-array", stride="3")
                    ET.SubElement(accessor, "param", name="X", type="float")
                    ET.SubElement(accessor, "param", name="Y", type="float")
                    ET.SubElement(accessor, "param", name="Z", type="float")

                    vertices = ET.SubElement(mesh, "vertices", id=f"point-{feature.GetFID()}-vertices")
                    ET.SubElement(vertices, "input", semantic="POSITION", source=f"#point-{feature.GetFID()}-positions")

                    # Add a primitive (e.g., a single point)
                    points = ET.SubElement(mesh, "points", count="1")
                    ET.SubElement(points, "input", semantic="VERTEX", source=f"#point-{feature.GetFID()}-vertices")

                # Extend with handling for LINESTRING, POLYGON, etc.

    # Write the DAE file
    tree = ET.ElementTree(root)
    tree.write(output_dae_path, encoding="utf-8", xml_declaration=True, pretty_print=True)
    print(f"Conversion complete. DAE file saved to {output_dae_path}")

# Example usage:
gml_to_dae("C:/Users/Isaac IK/Downloads/vm182013/VM18_2013.gml", "C:/Users/Isaac IK/Downloads/vm182013/output.dae")

ModuleNotFoundError: No module named 'osgeo'

In [8]:
from osgeo import ogr
from lxml import etree # For building XML (COLLADA)

def gml_to_dae(gml_file_path, dae_output_path):
    # 1. Parse GML data
    driver = ogr.GetDriverByName("GML")
    data_source = driver.Open(gml_file_path, 0) # 0 means read-only
    if data_source is None:
        print(f"Could not open {gml_file_path}")
        return

    layer = data_source.GetLayer(0) # Get the first layer

    # Initialize COLLADA document structure (simplified)
    root = etree.Element("COLLADA", version="1.4.1")
    library_geometries = etree.SubElement(root, "library_geometries")

    for feature in layer:
        geometry = feature.GetGeometryRef()
        if geometry:
            # Extract geometry data (e.g., points, lines, polygons)
            # This part requires detailed handling based on GML geometry types
            # and mapping them to COLLADA geometry elements (e.g., <mesh>, <vertices>, <polylist>)

            # Example for a point geometry (highly simplified)
            if geometry.GetGeometryName() == "POINT":
                point_x = geometry.GetX()
                point_y = geometry.GetY()
                point_z = geometry.GetZ() if geometry.GetZ() else 0.0

                geometry_element = etree.SubElement(library_geometries, "geometry", id=f"geometry_{feature.GetFID()}")
                mesh = etree.SubElement(geometry_element, "mesh")
                source = etree.SubElement(mesh, "source", id=f"positions_{feature.GetFID()}")
                float_array = etree.SubElement(source, "float_array", id=f"positions_array_{feature.GetFID()}", count="3")
                float_array.text = f"{point_x} {point_y} {point_z}"
                # ... add other COLLADA elements like <vertices>, <polylist> or <lines>

    # 3. Save the DAE file
    tree = etree.ElementTree(root)
    tree.write(dae_output_path, pretty_print=True, xml_declaration=True, encoding="utf-8")
    print(f"Conversion complete. DAE saved to {dae_output_path}")

# Example usage (replace with your actual file paths)
# gml_to_dae("input.gml", "output.dae")

ModuleNotFoundError: No module named 'osgeo'

In [11]:
from lxml import etree
import os
from datetime import datetime
import numpy as np

class CityGMLToDAEExporter:
    def __init__(self, input_gml, output_dir):
        self.input_gml = input_gml
        self.output_dir = output_dir
        
        # CityGML namespaces (supports both 1.0 and 2.0)
        self.namespaces = {
            'citygml': 'http://www.opengis.net/citygml/2.0',
            'bldg': 'http://www.opengis.net/citygml/building/2.0',
            'gml': 'http://www.opengis.net/gml',
            'gen': 'http://www.opengis.net/citygml/generics/2.0',
            'app': 'http://www.opengis.net/citygml/appearance/2.0',
            # Fallback to 1.0 namespaces
            'bldg1': 'http://www.opengis.net/citygml/building/1.0',
            'gen1': 'http://www.opengis.net/citygml/generics/1.0'
        }
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
    def parse_pos_list(self, pos_list_text):
        """Parse a gml:posList into coordinate tuples"""
        coords = list(map(float, pos_list_text.split()))
        points = [(coords[i], coords[i+1], coords[i+2] if i+2 < len(coords) else 0) 
                  for i in range(0, len(coords), 3)]
        return points
    
    def calculate_normal(self, v0, v1, v2):
        """Calculate face normal from three vertices"""
        edge1 = np.array([v1[0] - v0[0], v1[1] - v0[1], v1[2] - v0[2]])
        edge2 = np.array([v2[0] - v0[0], v2[1] - v0[1], v2[2] - v0[2]])
        
        normal = np.cross(edge1, edge2)
        length = np.linalg.norm(normal)
        
        if length > 0:
            normal = normal / length
        else:
            normal = np.array([0, 0, 1])
        
        return tuple(normal)
    
    def triangulate_polygon(self, points):
        """Convert a polygon into triangles using ear clipping method"""
        if len(points) < 3:
            return []
        
        if len(points) == 3:
            return [points]
        
        # Simple fan triangulation from first vertex
        triangles = []
        for i in range(1, len(points) - 1):
            triangles.append([points[0], points[i], points[i + 1]])
        
        return triangles
    
    def extract_building_geometry(self, building_elem):
        """Extract all geometry from a building element"""
        all_triangles = []
        
        # Find all polygons in the building
        polygons = building_elem.xpath('.//gml:Polygon', namespaces=self.namespaces)
        
        for polygon in polygons:
            # Get exterior ring
            pos_lists = polygon.xpath('.//gml:posList', namespaces=self.namespaces)
            
            for pos_list in pos_lists:
                if pos_list.text:
                    points = self.parse_pos_list(pos_list.text)
                    
                    # Remove duplicate consecutive points
                    cleaned_points = [points[0]]
                    for p in points[1:]:
                        if p != cleaned_points[-1]:
                            cleaned_points.append(p)
                    
                    # Remove last point if it's same as first (closed polygon)
                    if len(cleaned_points) > 1 and cleaned_points[0] == cleaned_points[-1]:
                        cleaned_points = cleaned_points[:-1]
                    
                    # Triangulate the polygon
                    if len(cleaned_points) >= 3:
                        triangles = self.triangulate_polygon(cleaned_points)
                        all_triangles.extend(triangles)
        
        return all_triangles
    
    def create_dae_file(self, building_elem, building_id):
        """Create a complete DAE file for a single building"""
        
        # Extract geometry as triangles
        triangles = self.extract_building_geometry(building_elem)
        
        if not triangles:
            print(f"Warning: No geometry found for building {building_id}")
            return None
        
        # Flatten all vertices and calculate normals
        vertices = []
        normals = []
        indices = []
        
        current_index = 0
        for tri in triangles:
            if len(tri) == 3:
                # Add vertices
                vertices.extend(tri)
                
                # Calculate normal for this triangle
                normal = self.calculate_normal(tri[0], tri[1], tri[2])
                # Each vertex gets the same normal (flat shading)
                normals.extend([normal, normal, normal])
                
                # Add indices
                indices.extend([current_index, current_index + 1, current_index + 2])
                current_index += 3
        
        # Create DAE XML structure
        dae_content = self.build_dae_xml(building_id, vertices, normals, indices)
        
        return dae_content
    
    def build_dae_xml(self, building_id, vertices, normals, indices):
        """Build complete DAE XML content"""
        
        # Prepare vertex positions
        positions = []
        for v in vertices:
            positions.extend([v[0], v[1], v[2]])
        
        # Prepare normals
        normal_values = []
        for n in normals:
            normal_values.extend([n[0], n[1], n[2]])
        
        vertex_count = len(vertices)
        triangle_count = len(indices) // 3
        
        # Format arrays
        position_str = ' '.join(f'{x:.6f}' for x in positions)
        normal_str = ' '.join(f'{x:.6f}' for x in normal_values)
        indices_str = ' '.join(map(str, indices))
        
        # Build DAE XML
        dae = f'''<?xml version="1.0" encoding="utf-8"?>
<COLLADA xmlns="http://www.collada.org/2005/11/COLLADASchema" version="1.4.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <asset>
    <contributor>
      <author>CityGML Converter</author>
      <authoring_tool>Python</authoring_tool>
    </contributor>
    <created>{datetime.now().isoformat()}</created>
    <modified>{datetime.now().isoformat()}</modified>
    <unit name="meter" meter="1"/>
    <up_axis>Z_UP</up_axis>
  </asset>
  
  <library_geometries>
    <geometry id="{building_id}-mesh" name="{building_id}">
      <mesh>
        <source id="{building_id}-positions">
          <float_array id="{building_id}-positions-array" count="{len(positions)}">{position_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-positions-array" count="{vertex_count}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
        
        <source id="{building_id}-normals">
          <float_array id="{building_id}-normals-array" count="{len(normal_values)}">{normal_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-normals-array" count="{len(normals)}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
        
        <vertices id="{building_id}-vertices">
          <input semantic="POSITION" source="#{building_id}-positions"/>
        </vertices>
        
        <triangles material="material0" count="{triangle_count}">
          <input semantic="VERTEX" source="#{building_id}-vertices" offset="0"/>
          <input semantic="NORMAL" source="#{building_id}-normals" offset="0"/>
          <p>{indices_str}</p>
        </triangles>
      </mesh>
    </geometry>
  </library_geometries>
  
  <library_materials>
    <material id="material0" name="material0">
      <instance_effect url="#effect0"/>
    </material>
  </library_materials>
  
  <library_effects>
    <effect id="effect0">
      <profile_COMMON>
        <technique sid="common">
          <phong>
            <diffuse>
              <color>0.8 0.8 0.8 1.0</color>
            </diffuse>
            <specular>
              <color>0.2 0.2 0.2 1.0</color>
            </specular>
            <shininess>
              <float>20.0</float>
            </shininess>
          </phong>
        </technique>
      </profile_COMMON>
    </effect>
  </library_effects>
  
  <library_visual_scenes>
    <visual_scene id="Scene" name="Scene">
      <node id="{building_id}" name="{building_id}" type="NODE">
        <matrix sid="transform">1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1</matrix>
        <instance_geometry url="#{building_id}-mesh" name="{building_id}">
          <bind_material>
            <technique_common>
              <instance_material symbol="material0" target="#material0"/>
            </technique_common>
          </bind_material>
        </instance_geometry>
      </node>
    </visual_scene>
  </library_visual_scenes>
  
  <scene>
    <instance_visual_scene url="#Scene"/>
  </scene>
</COLLADA>'''
        
        return dae
    
    def export_all_buildings(self):
        """Main function to export all buildings as individual DAE files"""
        try:
            # Parse the CityGML file
            tree = etree.parse(self.input_gml)
            root = tree.getroot()
            
            # Find all building elements (try both 2.0 and 1.0)
            buildings = root.xpath('//bldg:Building', namespaces=self.namespaces)
            
            if not buildings:
                # Try CityGML 1.0 namespace
                buildings = root.xpath('//bldg1:Building', namespaces=self.namespaces)
            
            if not buildings:
                # Try direct namespace approach for 2.0
                buildings = root.findall('.//{http://www.opengis.net/citygml/building/2.0}Building')
            
            if not buildings:
                # Try direct namespace approach for 1.0
                buildings = root.findall('.//{http://www.opengis.net/citygml/building/1.0}Building')
            
            print(f"Found {len(buildings)} buildings in the CityGML file")
            
            exported_count = 0
            failed_count = 0
            
            for idx, building in enumerate(buildings):
                # Get building ID
                building_id = building.get('{http://www.opengis.net/gml}id', f'Building_{idx}')
                
                # Sanitize filename
                safe_filename = "".join(c for c in building_id if c.isalnum() or c in ('-', '_'))
                output_file = os.path.join(self.output_dir, f"{safe_filename}.dae")
                
                print(f"Processing {idx + 1}/{len(buildings)}: {building_id}...")
                
                try:
                    # Create DAE content
                    dae_content = self.create_dae_file(building, building_id)
                    
                    if dae_content:
                        # Write to file
                        with open(output_file, 'w', encoding='utf-8') as f:
                            f.write(dae_content)
                        
                        exported_count += 1
                        print(f"  ✓ Exported to: {output_file}")
                    else:
                        failed_count += 1
                        print(f"  ✗ Failed: No geometry found")
                
                except Exception as e:
                    failed_count += 1
                    print(f"  ✗ Error: {str(e)}")
            
            # Summary
            print(f"\n{'='*60}")
            print(f"Export Summary:")
            print(f"  Total buildings: {len(buildings)}")
            print(f"  Successfully exported: {exported_count}")
            print(f"  Failed: {failed_count}")
            print(f"  Output directory: {self.output_dir}")
            print(f"{'='*60}")
            
            return exported_count, failed_count
        
        except Exception as e:
            print(f"Error processing CityGML file: {str(e)}")
            raise




In [12]:
# Usage example
if __name__ == "__main__":
    # Configure paths
    
    input_gml_file = "C:/Users/Isaac IK/Videos/VM14_2016.gml"  # Your CityGML file
    output_directory = "C:/Users/Isaac IK/Videos/exported_buildings_dae"  # Output folder for DAE files
 
    # Create exporter
    exporter = CityGMLToDAEExporter(input_gml_file, output_directory)
    
    # Export all buildings
    try:
        exported, failed = exporter.export_all_buildings()
        print(f"\nDone! Check the '{output_directory}' folder for your DAE files.")
        print("\nTo import into Blender:")
        print("  File → Import → Collada (.dae)")
        
    except Exception as e:
        print(f"Failed to export buildings: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Check that the input GML file path is correct")
        print("2. Ensure you have write permissions for the output directory")
        print("3. Install required packages: pip install lxml numpy")

Found 505 buildings in the CityGML file
Processing 1/505: Groupe9951527...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\Groupe9951527.dae
Processing 2/505: PC-13320...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\PC-13320.dae
Processing 3/505: 5573389...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\5573389.dae
Processing 4/505: PC-34670...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\PC-34670.dae
Processing 5/505: 5573388...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\5573388.dae
Processing 6/505: PC-32129...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\PC-32129.dae
Processing 7/505: 1885088...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\1885088.dae
Processing 8/505: 2316654...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae\2316654.dae
Processing 9/505: PC-08577...
  ✓ Exported to: C:/Users/Isaac IK/Videos/exported_buildings_dae

In [13]:
from lxml import etree
import os
from datetime import datetime
import numpy as np
import shutil

class CityGMLToDAEExporter:
    def __init__(self, input_gml, output_dir, texture_base_path=None):
        self.input_gml = input_gml
        self.output_dir = output_dir
        self.texture_base_path = texture_base_path or os.path.dirname(input_gml)
        
        # CityGML namespaces (supports both 1.0 and 2.0)
        self.namespaces = {
            'citygml': 'http://www.opengis.net/citygml/2.0',
            'bldg': 'http://www.opengis.net/citygml/building/2.0',
            'gml': 'http://www.opengis.net/gml',
            'gen': 'http://www.opengis.net/citygml/generics/2.0',
            'app': 'http://www.opengis.net/citygml/appearance/2.0',
            # Fallback to 1.0 namespaces
            'bldg1': 'http://www.opengis.net/citygml/building/1.0',
            'gen1': 'http://www.opengis.net/citygml/generics/1.0',
            'app1': 'http://www.opengis.net/citygml/appearance/1.0'
        }
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Create textures subdirectory
        self.textures_dir = os.path.join(output_dir, "textures")
        if not os.path.exists(self.textures_dir):
            os.makedirs(self.textures_dir)
    
    def parse_pos_list(self, pos_list_text):
        """Parse a gml:posList into coordinate tuples"""
        coords = list(map(float, pos_list_text.split()))
        points = [(coords[i], coords[i+1], coords[i+2] if i+2 < len(coords) else 0) 
                  for i in range(0, len(coords), 3)]
        return points
    
    def parse_texture_coordinates(self, tex_coord_text):
        """Parse texture coordinates from string"""
        coords = list(map(float, tex_coord_text.split()))
        # Group into (u, v) tuples
        tex_coords = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
        return tex_coords
    
    def extract_texture_info(self, building_elem):
        """Extract texture information from building appearance"""
        texture_info = {}
        
        # Find appearance elements
        appearances = building_elem.xpath('.//app:Appearance', namespaces=self.namespaces)
        if not appearances:
            appearances = building_elem.xpath('.//app1:Appearance', namespaces=self.namespaces)
        
        for appearance in appearances:
            # Find parameterized textures
            textures = appearance.xpath('.//app:ParameterizedTexture', namespaces=self.namespaces)
            if not textures:
                textures = appearance.xpath('.//app1:ParameterizedTexture', namespaces=self.namespaces)
            
            for texture in textures:
                # Get image URI
                image_uri_elem = texture.find('.//app:imageURI', namespaces=self.namespaces)
                if image_uri_elem is None:
                    image_uri_elem = texture.find('.//app1:imageURI', namespaces=self.namespaces)
                
                if image_uri_elem is not None and image_uri_elem.text:
                    image_uri = image_uri_elem.text.strip()
                    
                    # Find all targets (surface mappings)
                    targets = texture.findall('.//app:target', namespaces=self.namespaces)
                    if not targets:
                        targets = texture.findall('.//app1:target', namespaces=self.namespaces)
                    
                    for target in targets:
                        surface_id = target.get('uri', '').replace('#', '')
                        
                        # Get texture coordinates for this surface
                        tex_coord_elems = target.xpath('.//app:textureCoordinates', namespaces=self.namespaces)
                        if not tex_coord_elems:
                            tex_coord_elems = target.xpath('.//app1:textureCoordinates', namespaces=self.namespaces)
                        
                        for tex_coord_elem in tex_coord_elems:
                            ring_id = tex_coord_elem.get('ring', '').replace('#', '')
                            if tex_coord_elem.text:
                                tex_coords = self.parse_texture_coordinates(tex_coord_elem.text)
                                
                                if surface_id not in texture_info:
                                    texture_info[surface_id] = {
                                        'image': image_uri,
                                        'rings': {}
                                    }
                                texture_info[surface_id]['rings'][ring_id] = tex_coords
        
        return texture_info
    
    def copy_texture_file(self, image_uri, building_id):
        """Copy texture file to output directory and return relative path"""
        # Construct full path to source image
        source_path = os.path.join(self.texture_base_path, image_uri)
        
        if not os.path.exists(source_path):
            print(f"  Warning: Texture not found: {source_path}")
            return None
        
        # Get filename
        filename = os.path.basename(image_uri)
        
        # Create unique filename to avoid conflicts
        base_name, ext = os.path.splitext(filename)
        dest_filename = f"{building_id}_{base_name}{ext}"
        dest_path = os.path.join(self.textures_dir, dest_filename)
        
        # Copy file
        try:
            shutil.copy2(source_path, dest_path)
            print(f"  ✓ Copied texture: {filename}")
            return f"textures/{dest_filename}"
        except Exception as e:
            print(f"  Warning: Failed to copy texture: {e}")
            return None
    
    def calculate_normal(self, v0, v1, v2):
        """Calculate face normal from three vertices"""
        edge1 = np.array([v1[0] - v0[0], v1[1] - v0[1], v1[2] - v0[2]])
        edge2 = np.array([v2[0] - v0[0], v2[1] - v0[1], v2[2] - v0[2]])
        
        normal = np.cross(edge1, edge2)
        length = np.linalg.norm(normal)
        
        if length > 0:
            normal = normal / length
        else:
            normal = np.array([0, 0, 1])
        
        return tuple(normal)
    
    def triangulate_polygon(self, points, tex_coords=None):
        """Convert a polygon into triangles using fan triangulation"""
        if len(points) < 3:
            return []
        
        if len(points) == 3:
            triangles = [(points, tex_coords if tex_coords else None)]
            return triangles
        
        # Fan triangulation from first vertex
        triangles = []
        for i in range(1, len(points) - 1):
            tri_points = [points[0], points[i], points[i + 1]]
            tri_tex = None
            if tex_coords and len(tex_coords) >= len(points):
                tri_tex = [tex_coords[0], tex_coords[i], tex_coords[i + 1]]
            triangles.append((tri_points, tri_tex))
        
        return triangles
    
    def extract_building_geometry(self, building_elem, texture_info):
        """Extract all geometry from a building element with texture mapping"""
        all_triangles = []
        has_textures = False
        
        # Find all polygons in the building
        polygons = building_elem.xpath('.//gml:Polygon', namespaces=self.namespaces)
        
        for polygon in polygons:
            polygon_id = polygon.get('{http://www.opengis.net/gml}id', '')
            
            # Get exterior ring
            pos_lists = polygon.xpath('.//gml:posList', namespaces=self.namespaces)
            
            for pos_list in pos_lists:
                if pos_list.text:
                    points = self.parse_pos_list(pos_list.text)
                    
                    # Get the ring ID for texture mapping
                    parent_ring = pos_list.getparent()
                    ring_id = parent_ring.get('{http://www.opengis.net/gml}id', '') if parent_ring is not None else ''
                    
                    # Get texture coordinates if available
                    tex_coords = None
                    if polygon_id in texture_info and ring_id in texture_info[polygon_id]['rings']:
                        tex_coords = texture_info[polygon_id]['rings'][ring_id]
                        has_textures = True
                    
                    # Remove duplicate consecutive points
                    cleaned_points = [points[0]]
                    cleaned_tex = [tex_coords[0]] if tex_coords else None
                    
                    for idx, p in enumerate(points[1:], 1):
                        if p != cleaned_points[-1]:
                            cleaned_points.append(p)
                            if tex_coords and idx < len(tex_coords):
                                if cleaned_tex is None:
                                    cleaned_tex = []
                                cleaned_tex.append(tex_coords[idx])
                    
                    # Remove last point if it's same as first (closed polygon)
                    if len(cleaned_points) > 1 and cleaned_points[0] == cleaned_points[-1]:
                        cleaned_points = cleaned_points[:-1]
                        if cleaned_tex and len(cleaned_tex) > 1:
                            cleaned_tex = cleaned_tex[:-1]
                    
                    # Triangulate the polygon
                    if len(cleaned_points) >= 3:
                        triangles = self.triangulate_polygon(cleaned_points, cleaned_tex)
                        all_triangles.extend(triangles)
        
        return all_triangles, has_textures
    
    def create_dae_file(self, building_elem, building_id):
        """Create a complete DAE file for a single building with textures"""
        
        # Extract texture information
        texture_info = self.extract_texture_info(building_elem)
        
        # Copy texture files and get the main texture path
        texture_path = None
        if texture_info:
            # Get the first texture image
            for surface_id, info in texture_info.items():
                if 'image' in info:
                    texture_path = self.copy_texture_file(info['image'], building_id)
                    break
        
        # Extract geometry with texture coordinates
        triangles, has_textures = self.extract_building_geometry(building_elem, texture_info)
        
        if not triangles:
            print(f"Warning: No geometry found for building {building_id}")
            return None
        
        # Flatten all vertices, normals, and texture coordinates
        vertices = []
        normals = []
        tex_coords_list = []
        indices = []
        
        current_index = 0
        for tri_data in triangles:
            tri_points, tri_tex = tri_data
            
            if len(tri_points) == 3:
                # Add vertices
                vertices.extend(tri_points)
                
                # Calculate normal for this triangle
                normal = self.calculate_normal(tri_points[0], tri_points[1], tri_points[2])
                normals.extend([normal, normal, normal])
                
                # Add texture coordinates
                if tri_tex and len(tri_tex) == 3:
                    tex_coords_list.extend(tri_tex)
                else:
                    # Default UV coordinates if none provided
                    tex_coords_list.extend([(0, 0), (1, 0), (0, 1)])
                
                # Add indices
                indices.extend([current_index, current_index + 1, current_index + 2])
                current_index += 3
        
        # Create DAE XML structure
        dae_content = self.build_dae_xml(building_id, vertices, normals, tex_coords_list, indices, texture_path, has_textures)
        
        return dae_content
    
    def build_dae_xml(self, building_id, vertices, normals, tex_coords, indices, texture_path, has_textures):
        """Build complete DAE XML content with texture support"""
        
        # Prepare vertex positions
        positions = []
        for v in vertices:
            positions.extend([v[0], v[1], v[2]])
        
        # Prepare normals
        normal_values = []
        for n in normals:
            normal_values.extend([n[0], n[1], n[2]])
        
        # Prepare texture coordinates
        tex_coord_values = []
        for tc in tex_coords:
            tex_coord_values.extend([tc[0], tc[1]])
        
        vertex_count = len(vertices)
        triangle_count = len(indices) // 3
        
        # Format arrays
        position_str = ' '.join(f'{x:.6f}' for x in positions)
        normal_str = ' '.join(f'{x:.6f}' for x in normal_values)
        tex_coord_str = ' '.join(f'{x:.6f}' for x in tex_coord_values)
        
        # Build material/effect section
        material_section = ""
        effect_section = ""
        
        if texture_path and has_textures:
            # With texture
            material_section = f'''  <library_images>
    <image id="{building_id}-image" name="{building_id}-image">
      <init_from>{texture_path}</init_from>
    </image>
  </library_images>
  
  <library_materials>
    <material id="{building_id}-material" name="{building_id}-material">
      <instance_effect url="#{building_id}-effect"/>
    </material>
  </library_materials>'''
            
            effect_section = f'''  <library_effects>
    <effect id="{building_id}-effect">
      <profile_COMMON>
        <newparam sid="{building_id}-surface">
          <surface type="2D">
            <init_from>{building_id}-image</init_from>
          </surface>
        </newparam>
        <newparam sid="{building_id}-sampler">
          <sampler2D>
            <source>{building_id}-surface</source>
          </sampler2D>
        </newparam>
        <technique sid="common">
          <phong>
            <diffuse>
              <texture texture="{building_id}-sampler" texcoord="UVMap"/>
            </diffuse>
            <specular>
              <color>0.2 0.2 0.2 1.0</color>
            </specular>
            <shininess>
              <float>20.0</float>
            </shininess>
          </phong>
        </technique>
      </profile_COMMON>
    </effect>
  </library_effects>'''
        else:
            # Without texture - simple colored material
            material_section = f'''  <library_materials>
    <material id="{building_id}-material" name="{building_id}-material">
      <instance_effect url="#{building_id}-effect"/>
    </material>
  </library_materials>'''
            
            effect_section = f'''  <library_effects>
    <effect id="{building_id}-effect">
      <profile_COMMON>
        <technique sid="common">
          <phong>
            <diffuse>
              <color>0.8 0.8 0.8 1.0</color>
            </diffuse>
            <specular>
              <color>0.2 0.2 0.2 1.0</color>
            </specular>
            <shininess>
              <float>20.0</float>
            </shininess>
          </phong>
        </technique>
      </profile_COMMON>
    </effect>
  </library_effects>'''
        
        # Build texture coordinate source if we have textures
        texcoord_source = ""
        texcoord_input = ""
        if has_textures:
            texcoord_source = f'''
        <source id="{building_id}-texcoords">
          <float_array id="{building_id}-texcoords-array" count="{len(tex_coord_values)}">{tex_coord_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-texcoords-array" count="{len(tex_coords)}" stride="2">
              <param name="S" type="float"/>
              <param name="T" type="float"/>
            </accessor>
          </technique_common>
        </source>'''
            
            texcoord_input = f'''
          <input semantic="TEXCOORD" source="#{building_id}-texcoords" offset="0" set="0"/>'''
        
        # Build indices string
        indices_str = ' '.join(map(str, indices))
        
        # Build complete DAE XML
        dae = f'''<?xml version="1.0" encoding="utf-8"?>
<COLLADA xmlns="http://www.collada.org/2005/11/COLLADASchema" version="1.4.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <asset>
    <contributor>
      <author>CityGML Converter</author>
      <authoring_tool>Python with Textures</authoring_tool>
    </contributor>
    <created>{datetime.now().isoformat()}</created>
    <modified>{datetime.now().isoformat()}</modified>
    <unit name="meter" meter="1"/>
    <up_axis>Z_UP</up_axis>
  </asset>
  
{material_section}
  
{effect_section}
  
  <library_geometries>
    <geometry id="{building_id}-mesh" name="{building_id}">
      <mesh>
        <source id="{building_id}-positions">
          <float_array id="{building_id}-positions-array" count="{len(positions)}">{position_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-positions-array" count="{vertex_count}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
        
        <source id="{building_id}-normals">
          <float_array id="{building_id}-normals-array" count="{len(normal_values)}">{normal_str}</float_array>
          <technique_common>
            <accessor source="#{building_id}-normals-array" count="{len(normals)}" stride="3">
              <param name="X" type="float"/>
              <param name="Y" type="float"/>
              <param name="Z" type="float"/>
            </accessor>
          </technique_common>
        </source>
{texcoord_source}
        
        <vertices id="{building_id}-vertices">
          <input semantic="POSITION" source="#{building_id}-positions"/>
        </vertices>
        
        <triangles material="{building_id}-material" count="{triangle_count}">
          <input semantic="VERTEX" source="#{building_id}-vertices" offset="0"/>
          <input semantic="NORMAL" source="#{building_id}-normals" offset="0"/>{texcoord_input}
          <p>{indices_str}</p>
        </triangles>
      </mesh>
    </geometry>
  </library_geometries>
  
  <library_visual_scenes>
    <visual_scene id="Scene" name="Scene">
      <node id="{building_id}" name="{building_id}" type="NODE">
        <matrix sid="transform">1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1</matrix>
        <instance_geometry url="#{building_id}-mesh" name="{building_id}">
          <bind_material>
            <technique_common>
              <instance_material symbol="{building_id}-material" target="#{building_id}-material">
                <bind_vertex_input semantic="UVMap" input_semantic="TEXCOORD" input_set="0"/>
              </instance_material>
            </technique_common>
          </bind_material>
        </instance_geometry>
      </node>
    </visual_scene>
  </library_visual_scenes>
  
  <scene>
    <instance_visual_scene url="#Scene"/>
  </scene>
</COLLADA>'''
        
        return dae
    
    def export_all_buildings(self):
        """Main function to export all buildings as individual DAE files"""
        try:
            # Parse the CityGML file
            tree = etree.parse(self.input_gml)
            root = tree.getroot()
            
            # Find all building elements (try both 2.0 and 1.0)
            buildings = root.xpath('//bldg:Building', namespaces=self.namespaces)
            
            if not buildings:
                buildings = root.xpath('//bldg1:Building', namespaces=self.namespaces)
            
            if not buildings:
                buildings = root.findall('.//{http://www.opengis.net/citygml/building/2.0}Building')
            
            if not buildings:
                buildings = root.findall('.//{http://www.opengis.net/citygml/building/1.0}Building')
            
            print(f"Found {len(buildings)} buildings in the CityGML file")
            
            exported_count = 0
            failed_count = 0
            
            for idx, building in enumerate(buildings):
                # Get building ID
                building_id = building.get('{http://www.opengis.net/gml}id', f'Building_{idx}')
                
                # Sanitize filename
                safe_filename = "".join(c for c in building_id if c.isalnum() or c in ('-', '_'))
                output_file = os.path.join(self.output_dir, f"{safe_filename}.dae")
                
                print(f"Processing {idx + 1}/{len(buildings)}: {building_id}...")
                
                try:
                    # Create DAE content
                    dae_content = self.create_dae_file(building, building_id)
                    
                    if dae_content:
                        # Write to file
                        with open(output_file, 'w', encoding='utf-8') as f:
                            f.write(dae_content)
                        
                        exported_count += 1
                        print(f"  ✓ Exported to: {output_file}")
                    else:
                        failed_count += 1
                        print(f"  ✗ Failed: No geometry found")
                
                except Exception as e:
                    failed_count += 1
                    print(f"  ✗ Error: {str(e)}")
            
            # Summary
            print(f"\n{'='*60}")
            print(f"Export Summary:")
            print(f"  Total buildings: {len(buildings)}")
            print(f"  Successfully exported: {exported_count}")
            print(f"  Failed: {failed_count}")
            print(f"  Output directory: {self.output_dir}")
            print(f"  Textures directory: {self.textures_dir}")
            print(f"{'='*60}")
            
            return exported_count, failed_count
        
        except Exception as e:
            print(f"Error processing CityGML file: {str(e)}")
            raise




In [14]:
# Usage example
if __name__ == "__main__":
    # Configure paths
    input_gml_file = "C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/VM18_2016.gml"  # Your CityGML file
    output_directory =  "C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae"  # Output folder for DAE files
    texture_base_path = None  # Optional: specify where textures are (defaults to GML file directory)
    
    # Example: If your GML file references "VM14_2016_Appearance/Groupe9951527.jpg"
    # and the VM14_2016_Appearance folder is in the same directory as your GML file,
    # you don't need to set texture_base_path
    
    # Create exporter
    exporter = CityGMLToDAEExporter(input_gml_file, output_directory, texture_base_path)
    
    # Export all buildings
    try:
        exported, failed = exporter.export_all_buildings()
        print(f"\nDone! Check the '{output_directory}' folder for your DAE files.")
        print(f"Textures are in the '{output_directory}/textures' subfolder.")
        print("\nTo import into Blender:")
        print("  1. File → Import → Collada (.dae)")
        print("  2. The textures will be automatically linked")
        print("  3. Switch to 'Material Preview' or 'Rendered' view to see textures")
        
    except Exception as e:
        print(f"Failed to export buildings: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Check that the input GML file path is correct")
        print("2. Ensure texture images (JPG files) are in the correct location")
        print("3. Verify the texture path in the GML file matches your folder structure")
        print("4. Install required packages: pip install lxml numpy")

Found 307 buildings in the CityGML file
Processing 1/307: Groupe9365640...
  ✓ Copied texture: Groupe9365640.jpg
  ✓ Exported to: C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae\Groupe9365640.dae
Processing 2/307: Groupe9365637...
  ✓ Copied texture: Groupe9365637.jpg
  ✓ Exported to: C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae\Groupe9365637.dae
Processing 3/307: Groupe9365638...
  ✓ Copied texture: Groupe9365638.jpg
  ✓ Exported to: C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae\Groupe9365638.dae
Processing 4/307: Groupe9365539...
  ✓ Copied texture: Groupe9365539.jpg
  ✓ Exported to: C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae\Groupe9365539.dae
Processing 5/307: Groupe9365635...
  ✓ Copied texture: Groupe9365635.jpg
  ✓ Exported to: C:/Users/Isaac IK/Videos/New folder/VM18_2016_GML/exported_buildings_dae\Groupe9365635.dae
Processing 6/307: 1553308...
  ✓ Copied texture: 155330